# Optimise Prime

## Import libraries and dependencies


In [ ]:
#import hvplot.pandas
import panel as pn
pn.extension('plotly')

import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns


import optimiseprime.data_prep as dp
import optimiseprime.data_analysis as da
import optimiseprime.forecast as forecast
from optimiseprime.MCForecastTools import MCSimulation


import datetime as dt
import numpy as np
import pandas as pd
import yfinance as yf
import quantstats as qs
qs.extend_pandas()


# imports
import hvplot.pandas
import plotly.graph_objs as go  


%matplotlib inline


### Token Recommendations

In [ ]:
 
%%HTML
<div class='tableauPlaceholder' id='viz1641984633922' style='position: relative'><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='BarCharts1_16419846320250&#47;Multiple_Bar_Chart' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en-GB' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1641984633922');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

<div class='tableauPlaceholder' id='viz1641984513275' style='position: relative'><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='BarCharts_16419845132190&#47;Bar_Charts2' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en-GB' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1641984513275');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

<div class='tableauPlaceholder' id='viz1641983762340' style='position: relative'><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='path' value='views&#47;1H_16419837362410&#47;1H?:language=en-GB&amp;:embed=true&amp;publish=yes' /> <param name='toolbar' value='yes' /><param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en-GB' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1641983762340');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>
 

## Request user for portfolio details, and fetch price data from yfinance Yahoo! Finance API

In [ ]:
portfolio_choice = 0

while portfolio_choice not in [1, 2]:
    try:
        portfolio_choice = int(input(         
            "To analyse an existing portfolio, type 1\n"
            "To analyse a hypothetical portfolio, type 2\n"
        )
                              )
    # Print error message if type(portfolio_choice) != int                        
    except ValueError:
        print('Error: Invalid response.\n')
        
    # Print error message if portfolio choice not 1 or 2
    if portfolio_choice not in [1, 2]:
        print('Error: Invalid response.\n')

# Get portfolio data from user depending on choice:

ticker_list = []

while len(ticker_list) == 0:
    if portfolio_choice == 1:
        existing_portfolio = dp.get_existing_portfolio()
        
        # Create a list of only the tickers
        ticker_list = []
        for key, value in existing_portfolio.items():
            ticker_list.append(key)
        
    elif portfolio_choice == 2:
        ticker_list = dp.get_hypothetical_portfolio()
        # Request investment amount from user
        portfolio_value = dp.get_investment_amt()
        
    if len(ticker_list) == 0:
        print("You have not entered any tickers.")  

In [44]:
# Fetch data from yfinance for each ticker, and create pandas dataframe
portfolio_df = dp.get_ticker_data(ticker_list)
portfolio_df.dropna(inplace = True)
display(portfolio_df)


ETH-USD                                                      \
                   open         high          low        close       volume   
Date                                                                          
2019-07-27   219.617630   223.517258   204.903931   207.408310   6257703908   
2019-07-28   207.413773   212.041840   202.252899   211.186554   5504887934   
2019-07-29   211.123657   213.932053   208.764313   211.268051   5485190664   
2019-07-30   211.339203   213.614075   206.867615   210.522598   5489918859   
2019-07-31   210.492416   218.654144   210.437912   218.654144   6003828340   
...                 ...          ...          ...          ...          ...   
2022-01-13  3372.104980  3396.969727  3247.917480  3248.288574  15294466275   
2022-01-14  3248.648682  3330.766113  3203.823730  3310.001465  13562957230   
2022-01-15  3309.844238  3364.537842  3278.670898  3330.530762   9619999078   
2022-01-16  3330.387207  3376.401123  3291.563721  3350.921875   9505934874   
2022-01-17  3353.528809  3355.819336  3316.881592  3318.966309   9692304384   

                         LUNA1-USD                                   \
           daily_return       open       high        low      close   
Date                                                                  
2019-07-27    -0.055645   1.327436   1.402898   1.221938   1.310566   
2019-07-28     0.018216   1.310566   1.368595   1.236858   1.272989   
2019-07-29     0.000386   1.262270   1.375941   1.175443   1.294661   
2019-07-30    -0.003528   1.294435   1.374333   1.266484   1.294193   
2019-07-31     0.038626   1.285383   1.328408   1.265267   1.300102   
...                 ...        ...        ...        ...        ...   
2022-01-13    -0.036762  82.034729  83.770836  77.998466  77.998466   
2022-01-14     0.018999  78.055397  82.949577  76.755363  81.626823   
2022-01-15     0.006202  81.596352  87.800957  80.558029  87.574493   
2022-01-16     0.006122  87.512001  87.516914  84.797775  86.955864   
2022-01-17    -0.009536  87.012756  87.113663  85.617126  85.844223   

                                       
                  volume daily_return  
Date                                   
2019-07-27  6.033446e+06    -0.012709  
2019-07-28  1.643709e+06    -0.028672  
2019-07-29  3.966802e+06     0.017025  
2019-07-30  1.820705e+06    -0.000361  
2019-07-31  1.155167e+06     0.004566  
...                  ...          ...  
2022-01-13  2.445146e+09    -0.049607  
2022-01-14  1.828747e+09     0.046518  
2022-01-15  1.586855e+09     0.072864  
2022-01-16  1.423445e+09    -0.007064  
2022-01-17  1.342441e+09    -0.012784  

[906 rows x 12 columns]

In [45]:
# Keep only tickers in ticker_list for which data is available
ticker_list = [ticker for ticker in list(portfolio_df.columns.levels[0])]
value_list = []
weight_list = []

# Print portfolio data for visual confirmation
if portfolio_choice == 1:    
    # Calculate portfolio value of each cryptocurrency held
    for ticker in existing_portfolio:
        existing_portfolio[ticker].append(
            {'value': portfolio_df[ticker].iloc[-1, 3] * existing_portfolio[ticker][0]['units']})
    print(f"--------------------------")                      
    print(f"Existing Portfolio:")
    portfolio_value = 0
    for ticker, units in existing_portfolio.items():
        value = existing_portfolio[ticker][1]['value']
        print(f"Value of {existing_portfolio[ticker][0]['units']} {ticker}: ${value:.2f}")
        portfolio_value += value
       
    # Add weights for each crypto to existing_portfolio
    for ticker, units in existing_portfolio.items():
        weight = existing_portfolio[ticker][1]['value'] / portfolio_value
        existing_portfolio[ticker][1]['weight'] = weight

    print(f"\nTotal portfolio value: ${portfolio_value:.2f}\n")       


    #find current weights
    print("Crypto Weights")
    print(f"The portfolio holds:")
    for ticker, units in existing_portfolio.items():
        weight_list.append(existing_portfolio[ticker][1]['weight'])
        value_list.append(existing_portfolio[ticker][1]['value'])
    pie_df = px.data.tips()
    px.pie(pie_df, values=value_list, names=ticker_list,   title="Distribution of existing portfolio", color_discrete_sequence=px.colors.sequential.RdBu).show()

elif portfolio_choice == 2:
    print(
        f"--------------------------\n"
        f"Hypothetical Portfolio:\n"
        f"{[ticker.replace('-USD', '') for ticker in ticker_list]}\n"                   
        f"Investment amount:\n"
        f"${portfolio_value:.2f}\n")
print(
    f"NOTE:\n"
    f"To achieve a fair comparison of risk-reward ratios, historical price data will be retrieved from earliest date for which ALL cryptocurrencies specified are available.\n"
    f"While this ensures fair comparison of risk-reward metrics, it may compromise accuracy of these metrics if the sample sizes of historical price data are reduced.\n"
    f"Earliest date for which price data is available for all cryptocurrencies in your portfolio: {dt.datetime.date(portfolio_df.index[0])}"
)
print(f"--------------------------")





--------------------------
Hypothetical Portfolio:
['ETH', 'LUNA1']
Investment amount:
$30000.00

NOTE:
To achieve a fair comparison of risk-reward ratios, historical price data will be retrieved from earliest date for which ALL cryptocurrencies specified are available.
While this ensures fair comparison of risk-reward metrics, it may compromise accuracy of these metrics if the sample sizes of historical price data are reduced.
Earliest date for which price data is available for all cryptocurrencies in your portfolio: 2019-07-27
--------------------------


## Visualisation of Portfolio Historical Returns

In [46]:
#weights of original
if portfolio_choice == 1:
    weight_og = pd.DataFrame([weight_list], columns = ticker_list)

In [47]:
daily_returns = portfolio_df.filter(like = "daily_return")
dr = []
for ticker in ticker_list:
    dr.append(ticker)
daily_returns.columns = dr

In [48]:
#FIRST PAGE 
title = "Analysis of Daily Returns of Cryptos"
welcome_text = "In order to determine the sharpe ratio, the daily returns need to be known. \n Here the returns can be visually analysed to view returns and risk" 
daily_returns_plot = px.line(daily_returns, x = daily_returns.index, y = daily_returns.columns, title = "daily returns (volatility)")
first_page = pn.Column(title, welcome_text, daily_returns.head(), daily_returns_plot)

#SECOND PAGE
cumulative_returns = (1 + daily_returns).cumprod() - 1
cumul_plot = px.line(cumulative_returns, x = cumulative_returns.index, y = cumulative_returns.columns, title = "Cumulative Returns")

second_page = pn.Column("cumulative returns overtime", cumulative_returns.head(), cumul_plot)

#THIRD PAGE
boxplot = px.box(daily_returns, y=daily_returns.columns, title = 'Visual Representation of Risk')
third_page = pn.Column("Risk visualised on a box plot", boxplot)

#Fourth Page
if portfolio_choice == 1:
    portfolio_og = daily_returns.dot(weight_og.iloc[0,:])
    portfolio_og = pd.DataFrame(portfolio_og, columns = ['My Portfolio'])

    cumulative_returns_port = (1 + portfolio_og).cumprod() - 1
    daily_returns_port = px.line(portfolio_og, x = portfolio_og.index, y = portfolio_og.columns, title = "Daily returns of my portfolio")
    cumul_plot_port = px.line(cumulative_returns_port, x = cumulative_returns_port.index, y = cumulative_returns_port.columns, title = "Cumulative Returns of my portfolio")

    fourth_page = pn.Column("Portfolio Analysis", portfolio_og.head(), daily_returns_port, cumul_plot_port)

if portfolio_choice == 1:
    daily_dashboard = pn.Tabs(
        ("Daily Returns of tickers", first_page),
        ("Cumulative Returns of tickers", second_page),
        ("Risk of tickers", third_page),
        ("Historical Returns of my portfolio", fourth_page)
    )
else:
    daily_dashboard = pn.Tabs(
    ("Daily Returns of tickers", first_page),
    ("Cumulative Returns of tickers", second_page),
    ("Risk of tickers", third_page)
)


daily_dashboard.show()

Launching server at http://localhost:53408


---

# Analysis: Risk-Reward Ratios and Recommended Weights

### Calculate ratios:
* Sharpe ratio
* Sortino ratio
* Adjusted sortino ratio
* Gain to Pain ratio

In [49]:
# Calculate each of the following risk-reward ratio types
sharpe = da.calculate_sharpe_ratio(ticker_list, portfolio_df)
sortino =  da.calculate_sortino_ratio(ticker_list, portfolio_df)
adjusted_sortino = da.calculate_adjusted_sortino(ticker_list, portfolio_df)
gain_pain_ratio = da.calculate_gain_pain_ratio(ticker_list, portfolio_df)


# Store all ratios into a dict
ratios_df = pd.DataFrame(
    {
    'sharpe': sharpe,
    'sortino': sortino,
    'adj_sortino': adjusted_sortino,
    'gain_pain': gain_pain_ratio,
    }
)
#display(ratios_df)

# Calculate proportion scores for each risk-reward metric
weights = da.calculate_weights(ratios_df)

# If user has selected to analyse an existing portfolio, add user's portfolio weights as well.
if portfolio_choice == 1 :
    weight_og = weight_og.transpose()
    weights = weights.merge(weight_og, left_index=True, right_index=True)
weights=weights.rename(columns = {0:'user_portfolio'})
weights



,sharpe,sortino,adj_sortino,gain_pain
ETH-USD,0.480149,0.417898,0.417884,0.445092
LUNA1-USD,0.519851,0.582102,0.582116,0.554908


In [50]:
print(
    f"Portfolio allocation recommendations\n"
    f"Based on historical returns from {dt.datetime.date(portfolio_df.index[0])} to {dt.datetime.date(portfolio_df.index[-1])}"
)
print(f"Total portfolio value: ${portfolio_value:.2f}")
print(f"============================================================="
)

# Present all ratios in descending order

for column in ratios_df:
    if column == 'sharpe':
        da.sharpe_portfolio(ratios_df, weights, portfolio_value)
    elif column == 'sortino':
        da.sortino_portfolio(ratios_df, weights, portfolio_value)
    elif column == 'adj_sortino':
        da.adj_sortino_portfolio(ratios_df, weights, portfolio_value)
    elif column == 'gain_pain':
        da.gain_pain_portfolio(ratios_df, weights, portfolio_value)
        
if portfolio_choice == 1:
    user_portfolio = da.user_portfolio(weights, portfolio_value)


Portfolio allocation recommendations
Based on historical returns from 2019-07-27 to 2022-01-17
Total portfolio value: $30000.00

Metric: Sharpe Ratio
--------------
LUNA1-USD                          1.78 
Recommended % of total portfolio   51.99%
Recommended value allocation       $15595.54

ETH-USD                            1.64 
Recommended % of total portfolio   48.01%
Recommended value allocation       $14404.46


Metric: Sortino Ratio
--------------
LUNA1-USD                          3.36 
Recommended % of total portfolio   58.21%
Recommended value allocation       $17463.07

ETH-USD                            2.42 
Recommended % of total portfolio   41.79%
Recommended value allocation       $12536.93


Metric: Adjusted Sortino Ratio
--------------
LUNA1-USD                          2.38 
Recommended % of total portfolio   58.21%
Recommended value allocation       $17463.49

ETH-USD                            1.71 
Recommended % of total portfolio   41.79%
Recommended value allo

## Preparation of Data for Visualisations of Risk-Reward Metrics 

In [51]:
# Transpose data
weights= weights.transpose()
weights

,ETH-USD,LUNA1-USD
sharpe,0.480149,0.519851
sortino,0.417898,0.582102
adj_sortino,0.417884,0.582116
gain_pain,0.445092,0.554908


In [52]:
names = weights.values.tolist()

sharpe_pie = px.pie(weights_transposed, 
    names = ticker_list,
    values = names[0],
    title = "Sharpe Recommended Weights")

sortino_pie =px.pie(weights_transposed, 
    names = ticker_list,
    values = names[1],
    title = "Sortino Recommended Weights")

adj_sortino_pie = px.pie(weights_transposed, 
    names = ticker_list,
    values = names[0],
    title = "Adj Sortino Recommended Weights")

pain_gain_pie = px.pie(weights_transposed, 
    names = ticker_list,
    values = names[1],
    title = "Pain Gain Recommended Weights")


if portfolio_choice == 1:
        px.pie(pie_df, values=value_list, names=ticker_list,   title="Distribution of existing portfolio", color_discrete_sequence=px.colors.sequential.RdBu)


In [53]:
daily_returns

,ETH-USD,LUNA1-USD
Date,,
2019-07-27,-0.055645,-0.012709
2019-07-28,0.018216,-0.028672
2019-07-29,0.000386,0.017025
2019-07-30,-0.003528,-0.000361
2019-07-31,0.038626,0.004566
...,...,...
2022-01-13,-0.036762,-0.049607
2022-01-14,0.018999,0.046518
2022-01-15,0.006202,0.072864


In [54]:
weights = weights.transpose()

In [55]:
# Get portfolio value over time, multiplied by weights, and store in dataframe
portfolio_returns_list = []
port_daily_returns=[]
for column_name in weights.columns:
    weights_list = weights[column_name].tolist()
    portfolio_returns = daily_returns.dot(weights_list)
    portfolio_returns = portfolio_returns.rename(column_name)
    portfolio_cumul_returns = (1 + portfolio_returns).cumprod() - 1
    portfolio_returns_list.append(portfolio_cumul_returns)
    port_daily_returns.append(portfolio_returns)

    
port_returns_df = pd.DataFrame(portfolio_returns_list)
port_daily_returns = pd.DataFrame(port_daily_returns)
port_returns_df = port_returns_df.transpose()
port_daily_returns = port_daily_returns.transpose()





---

# Forecasting

## Autoregression Forecasting

In [56]:

true_false = pn.widgets.Select(name='View whole graph', options=[True, False])

    
interact = pn.interact(
    forecast.forecast_all_portfolios, 
    portfolio_returns_list = [portfolio_returns_list[0:5]], 
    window = range(365), 
    foward_looking = range(365), 
    full_picture = true_false
)




## Monte Carlo Simulation

In [57]:
#Set number of simulations 
num_sims = 500
num_trading_days = 365 * (int(input("Key in number of years to forecast:")))

# Run simulations for each strategy with number of trading days specified
MC_sharpe = MCSimulation(
    portfolio_data = portfolio_df,
    num_simulation = num_sims,
    weights = weights['sharpe'].tolist(),
    num_trading_days = num_trading_days
)

MC_sortino = MCSimulation(
    portfolio_data = portfolio_df,
    num_simulation = num_sims,
    weights = weights['sortino'].tolist(),
    num_trading_days = num_trading_days
)

MC_adj_sortino = MCSimulation(
    portfolio_data = portfolio_df,
    num_simulation = num_sims,
    weights = weights['adj_sortino'].tolist(),
    num_trading_days = num_trading_days
)

MC_gain_pain = MCSimulation(
    portfolio_data = portfolio_df,
    num_simulation = num_sims,
    weights = weights['gain_pain'].tolist(),
    num_trading_days = num_trading_days
)

simulations_list = [MC_sharpe, MC_sortino, MC_adj_sortino, MC_gain_pain]
if portfolio_choice == 1:
    MC_user = MCSimulation(
        portfolio_data = portfolio_df,
        num_simulation = num_sims,
        weights = weights['user_portfolio'].tolist(),
        num_trading_days = num_trading_days
    )
    simulations_list.append(MC_user)

for simulation in simulations_list:
    simulation.calc_cumulative_return()

Key in number of years to forecast: 1


Running Monte Carlo simulation number 0.
Running Monte Carlo simulation number 10.
Running Monte Carlo simulation number 20.
Running Monte Carlo simulation number 30.
Running Monte Carlo simulation number 40.
Running Monte Carlo simulation number 50.
Running Monte Carlo simulation number 60.
Running Monte Carlo simulation number 70.
Running Monte Carlo simulation number 80.
Running Monte Carlo simulation number 90.
Running Monte Carlo simulation number 100.


C:\Users\colle\Documents\FinTech\project final\Group1-Project1-main\optimiseprime\MCForecastTools.py:116: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`



Running Monte Carlo simulation number 110.
Running Monte Carlo simulation number 120.
Running Monte Carlo simulation number 130.
Running Monte Carlo simulation number 140.
Running Monte Carlo simulation number 150.
Running Monte Carlo simulation number 160.
Running Monte Carlo simulation number 170.
Running Monte Carlo simulation number 180.
Running Monte Carlo simulation number 190.
Running Monte Carlo simulation number 200.
Running Monte Carlo simulation number 210.
Running Monte Carlo simulation number 220.
Running Monte Carlo simulation number 230.
Running Monte Carlo simulation number 240.
Running Monte Carlo simulation number 250.
Running Monte Carlo simulation number 260.
Running Monte Carlo simulation number 270.
Running Monte Carlo simulation number 280.
Running Monte Carlo simulation number 290.
Running Monte Carlo simulation number 300.
Running Monte Carlo simulation number 310.
Running Monte Carlo simulation number 320.
Running Monte Carlo simulation number 330.
Running Mon

In [58]:
# Compute summary statistics from the simulated daily returns, for each simulation

all_cumulative_pnl = []
for simulation in simulations_list:
    simulated_returns = pd.DataFrame(
        {
            "mean": list(simulation.simulated_return.mean(axis=1)),
            "median": list(simulation.simulated_return.median(axis=1)),            
            "min": list(simulation.simulated_return.min(axis=1)),
            "max": list(simulation.simulated_return.max(axis=1))
            
        }
    )
    all_cumulative_pnl.append(simulated_returns)

# Concatenate cumulative pnls predicted by MC simulations for each strategy
all_mean_cumulative_pnl = pd.concat([df['mean'] for df in all_cumulative_pnl], join = 'inner', axis = 'columns') 

# Multiply by user's total portfolio value
all_mean_cumulative_pnl = all_mean_cumulative_pnl * portfolio_value
# Rename columns
all_mean_cumulative_pnl.columns = weights.columns.tolist()


In [59]:
# Visualise mean projection of each portfolio

sim_port_mean = px.line(
    all_mean_cumulative_pnl,
    title = f"Simulated mean portfolio values over the next {num_trading_days / 365} years by R-R strategies",
    labels = {
        "value": "Portfolio Value ($)",
        "index": "Number of Trading Days"
        
    }
)


## Dashboard - Visualisations of Portfolio Weightages and Recommendations 

In [63]:
#FIRST PAGE  - what was done, how
title = "Analysis of Portfolios"
welcome_text = "What was done and how" 
first_page = pn.Column(title, welcome_text)

#SECOND PAGE - returns of tickers
daily_returns = daily_returns
cumulative_returns = (1 + daily_returns).cumprod() - 1
cumul_plot = px.line(cumulative_returns, x = cumulative_returns.index, y = cumulative_returns.columns, title = "Cumulative Returns of Tickers")
daily_returns_plot = px.line(daily_returns, x = daily_returns.index, y = daily_returns.columns, title = "Daily Returns of Tickers")
boxplot = px.box(daily_returns, y=daily_returns.columns, title = 'Risk of Tickers')

second_page = pn.Column("Ticker Analysis", daily_returns.head(), daily_returns_plot, cumulative_returns.head(), cumul_plot)

#THIRD PAGE (if) - og port analysis
if portfolio_choice ==1:
    pie_df = px.data.tips()
    pie_og = px.pie(pie_df, values=value_list, names=ticker_list, title="distribution of existing portfolio", color_discrete_sequence=px.colors.sequential.RdBu)
    third_page = pn.Column("Original Portfolio Analysis", weight_og, pie_og, portfolio_og.head(), daily_returns_port, cumul_plot_port, boxplot)
        
#Fourth Page
if portfolio_choice == 1:
    fourth_page = pn.Column("Recommended Weights in Comparison to Original Portfolio",  weight_og, pie_og, weights, sharpe_pie, sortino_pie, adj_sortino_pie, pain_gain_pie)
else:
    fourth_page = pn.Column("Recommended Weights for Portfolio", weights, sharpe_pie, sortino_pie, adj_sortino_pie, pain_gain_pie)
        
#Fifth Page
port_returns_plot = px.line(port_daily_returns, x = port_daily_returns.index, y = port_daily_returns.columns, title = "Daily Returns of Portfolio")
port_boxplot = px.box(port_daily_returns, y=port_daily_returns.columns, title = 'Risk of Portfolio')
fifth_page = pn.Column("Portfolio Analysis", "Portfolio Returns", port_daily_returns.tail(), port_returns_plot, "Portfolio Cumulative Returns", port_returns_df.head(), interact, "Visualised Risk of Each Portfolio", port_boxplot)

#sixth
ann_std = port_daily_returns.std() * np.sqrt(252)
# Calculate rolling standard deviation
r21_std = port_daily_returns.rolling(window=21).std()
r21_std_plot = px.line(r21_std, x = r21_std.index, y = r21_std.columns, title = "Rolling-21 Day Std")

correlation = port_daily_returns.corr()
corr_plot = px.imshow(correlation)

sixth_page = pn.Column("Annual Standard Deviation (annualised volatility)", ann_std, r21_std_plot, "Correlation of portfolios", correlation, corr_plot, "portfolio Close", port_daily_returns.tail(), "Portfolio Returns", port_returns_df.tail())

seventh_page = pn.Column("Simulated Returns using Monte Carlo simulations", simulated_returns.tail(), "Mean Cumulative PNL", all_mean_cumulative_pnl.tail(), sim_port_mean)


if portfolio_choice == 1:
    final_dashboard = pn.Tabs(
        ("Daily Returns of tickers", first_page),
        ("Cumulative Returns of tickers", second_page),
        ("Risk of tickers", third_page),
        ("Recommended Weights", fourth_page),
        ("Comparing Portfolios", fifth_page),
        ("Comparing Portfolios" , sixth_page),
        ("Monte Carlo" , seventh_page)
    )
else:
    final_dashboard = pn.Tabs(
    ("Welcome", first_page),
    ("Ticker Analysis", second_page),
    ("Recommended Weights", fourth_page),
    ("Comparing Portfolios", fifth_page),
    ("Comparing Portfolios" , sixth_page),
    ("Monte Carlo" , seventh_page)
)

final_dashboard.show()
    


Launching server at http://localhost:53430
